In [1]:
BATCH_SIZE = 32

In [2]:
import os
import cv2
import torch
import numpy as np

In [3]:
def load_data(img_size=112):
    data = []
    index = -1
    labels = {}
    for directory in os.listdir('./data/'):
        index += 1
        labels[f'./data/{directory}/'] = [index,-1]
    print(len(labels))
    for label in labels:
        for file in os.listdir(label):
            filepath = label + file
            img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(img_size,img_size))
            img = img / 255.0
            data.append([
                np.array(img),
                labels[label][0]
            ])
            labels[label][1] += 1
    for _ in range(12):
        np.random.shuffle(data)
    print(len(data))
    np.save('./data.npy',data)
    return data

In [4]:
import torch

In [5]:
def other_loading_data_proccess(data):
    X = []
    y = []
    print('going through the data..')
    for d in data:
        X.append(d[0])
        y.append(d[1])
    print('splitting the data')
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    print('turning data to tensors')
    X_train = torch.from_numpy(np.array(X_train))
    y_train = torch.from_numpy(np.array(y_train))
    X_test = torch.from_numpy(np.array(X_test))
    y_test = torch.from_numpy(np.array(y_test))
    return [X_train,X_test,y_train,y_test]

In [6]:
REBUILD_DATA = True
if REBUILD_DATA:
    data = load_data()
    np.random.shuffle(data)
    X_train,X_test,y_train,y_test = other_loading_data_proccess(data)

In [7]:
import torch
import torch.nn as nn

In [8]:
import torch.nn.functional as F

In [9]:
# class Test_Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 25 * 25, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 36)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 25 * 25)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [10]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 25 * 25, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84, 36)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 25 * 25)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
device = torch.device('cuda')

In [12]:
model = Test_Model().to(device)

In [13]:
# preds = model(X_test.reshape(-1,1,112,112).float())

In [14]:
# preds[0]

In [15]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [16]:
EPOCHS = 3

In [17]:
loss_logs = []

In [18]:
from tqdm import tqdm

In [19]:
PROJECT_NAME = "Sign-Language-Recognition"

In [20]:
def test(net,X,y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in range(len(X)):
            real_class = torch.argmax(y[i]).to(device)
            net_out = net(X[i].view(-1,1,112,112).to(device).float())
            net_out = net_out[0]
            predictied_class = torch.argmax(net_out)
            if predictied_class == real_class:
                correct += 1
            total += 1
    return round(correct/total,3)

In [21]:
import wandb

In [22]:
len(os.listdir('./data/'))

36

In [23]:
import random

In [24]:
index = random.randint(0,29)
print(index)
wandb.init(project=PROJECT_NAME,name='test')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item(),'accuracy':test(model,X_train,y_train)*100,'val_accuracy':test(model,X_test,y_test)*100,'pred':torch.argmax(preds[index]),'real':torch.argmax(y_batch[index])})
wandb.finish()

In [25]:
import matplotlib.pyplot as plt

In [26]:
import pandas as pd

In [27]:
df = pd.Series(loss_logs)

In [28]:
df.plot.line(figsize=(12,6))

<AxesSubplot:>

In [29]:
test(model,X_test,y_test)

0.021

In [30]:
test(model,X_train,y_train)

0.026

In [31]:
preds

tensor([[ 3.9254, -0.6911,  4.3276,  ...,  5.7012, -1.4525, -2.0458],
        [ 0.5794,  0.6124, -2.6371,  ..., -0.7431,  0.7737,  1.9322],
        [ 1.4467,  0.8766, -2.9301,  ..., -0.1083,  4.6229, -0.0687],
        ...,
        [ 3.6922, -2.1974, -1.4462,  ...,  2.7416,  4.2992, -1.4075],
        [ 0.8374, -0.8080,  0.1401,  ..., -1.1702, -2.9645, -0.1229],
        [ 1.9989,  2.4674,  4.3512,  ...,  4.8605, -1.7597, -2.1128]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [32]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for i in range(len(X_test)):
        real_class = torch.argmax(y_test[i]).to(device)
        net_out = model(X_test[i].view(-1,1,112,112).to(device).float())
        net_out = net_out[0]
        predictied_class = torch.argmax(net_out)
#         print(predictied_class)
        if str(predictied_class) == str(real_class):
            correct += 1
        total += 1
print(round(correct/total,3))

In [33]:
total

628

In [34]:
for real,pred in zip(y_batch,preds):
    print(real)
    print(torch.argmax(pred))
    print('\n')

In [35]:
BATCH_SIZE = 32

In [36]:
import os
import cv2
import torch
import numpy as np

In [37]:
def load_data(img_size=112):
    data = []
    index = -1
    labels = {}
    for directory in os.listdir('./data/'):
        index += 1
        labels[f'./data/{directory}/'] = [index,-1]
    print(len(labels))
    for label in labels:
        for file in os.listdir(label):
            filepath = label + file
            img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(img_size,img_size))
            img = img / 255.0
            data.append([
                np.array(img),
                labels[label][0]
            ])
            labels[label][1] += 1
    for _ in range(12):
        np.random.shuffle(data)
    print(len(data))
    np.save('./data.npy',data)
    return data

In [38]:
import torch

In [39]:
def other_loading_data_proccess(data):
    X = []
    y = []
    print('going through the data..')
    for d in data:
        X.append(d[0])
        y.append(d[1])
    print('splitting the data')
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    print('turning data to tensors')
    X_train = torch.from_numpy(np.array(X_train))
    y_train = torch.from_numpy(np.array(y_train))
    X_test = torch.from_numpy(np.array(X_test))
    y_test = torch.from_numpy(np.array(y_test))
    return [X_train,X_test,y_train,y_test]

In [40]:
REBUILD_DATA = True
if REBUILD_DATA:
    data = load_data()
    np.random.shuffle(data)
    X_train,X_test,y_train,y_test = other_loading_data_proccess(data)

In [41]:
import torch
import torch.nn as nn

In [42]:
import torch.nn.functional as F

In [43]:
# class Test_Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 25 * 25, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 36)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 25 * 25)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [44]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 25 * 25, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 36)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 25 * 25)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [45]:
device = torch.device('cuda')

In [46]:
model = Test_Model().to(device)

In [47]:
# preds = model(X_test.reshape(-1,1,112,112).float())

In [48]:
# preds[0]

In [49]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [50]:
EPOCHS = 3

In [51]:
loss_logs = []

In [52]:
from tqdm import tqdm

In [53]:
PROJECT_NAME = "Sign-Language-Recognition"

In [54]:
def test(net,X,y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in range(len(X)):
            real_class = torch.argmax(y[i]).to(device)
            net_out = net(X[i].view(-1,1,112,112).to(device).float())
            net_out = net_out[0]
            predictied_class = torch.argmax(net_out)
            if predictied_class == real_class:
                correct += 1
            total += 1
    return round(correct/total,3)

In [55]:
import wandb

In [56]:
len(os.listdir('./data/'))

36

In [57]:
import random

In [58]:
index = random.randint(0,29)
print(index)
wandb.init(project=PROJECT_NAME,name='test')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item(),'accuracy':test(model,X_train,y_train)*100,'val_accuracy':test(model,X_test,y_test)*100,'pred':torch.argmax(preds[index]),'real':torch.argmax(y_batch[index])})
wandb.finish()

In [59]:
import matplotlib.pyplot as plt

In [60]:
import pandas as pd

In [61]:
df = pd.Series(loss_logs)

In [62]:
df.plot.line(figsize=(12,6))

<AxesSubplot:>

In [63]:
test(model,X_test,y_test)

0.008

In [64]:
test(model,X_train,y_train)

0.011

In [65]:
preds

tensor([[ 3.2202,  3.4554,  5.2178,  ...,  2.7196, -0.8697, -3.1623],
        [ 0.5938,  2.2880, -0.5461,  ...,  5.8734,  3.3813, -1.8309],
        [ 1.6170, -1.7326, -0.1594,  ...,  0.0228, -0.8004, -1.0772],
        ...,
        [ 1.8339, -2.8979, -1.8774,  ..., -0.5880,  1.8611, -0.1998],
        [ 1.8347, -2.5374, -1.2612,  ..., -1.3733, -1.2941, -0.4215],
        [ 4.1641, -2.3633, -0.4983,  ...,  0.4744,  1.1293, -1.9589]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [66]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for i in range(len(X_test)):
        real_class = torch.argmax(y_test[i]).to(device)
        net_out = model(X_test[i].view(-1,1,112,112).to(device).float())
        net_out = net_out[0]
        predictied_class = torch.argmax(net_out)
#         print(predictied_class)
        if str(predictied_class) == str(real_class):
            correct += 1
        total += 1
print(round(correct/total,3))

In [67]:
total

628

In [68]:
for real,pred in zip(y_batch,preds):
    print(real)
    print(torch.argmax(pred))
    print('\n')

In [69]:
NAME = ""

In [70]:
BATCH_SIZE = 32

In [71]:
import os
import cv2
import torch
import numpy as np

In [72]:
def load_data(img_size=112):
    data = []
    index = -1
    labels = {}
    for directory in os.listdir('./data/'):
        index += 1
        labels[f'./data/{directory}/'] = [index,-1]
    print(len(labels))
    for label in labels:
        for file in os.listdir(label):
            filepath = label + file
            img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(img_size,img_size))
            img = img / 255.0
            data.append([
                np.array(img),
                labels[label][0]
            ])
            labels[label][1] += 1
    for _ in range(12):
        np.random.shuffle(data)
    print(len(data))
    np.save('./data.npy',data)
    return data

In [73]:
import torch

In [74]:
def other_loading_data_proccess(data):
    X = []
    y = []
    print('going through the data..')
    for d in data:
        X.append(d[0])
        y.append(d[1])
    print('splitting the data')
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    print('turning data to tensors')
    X_train = torch.from_numpy(np.array(X_train))
    y_train = torch.from_numpy(np.array(y_train))
    X_test = torch.from_numpy(np.array(X_test))
    y_test = torch.from_numpy(np.array(y_test))
    return [X_train,X_test,y_train,y_test]

In [75]:
REBUILD_DATA = True
if REBUILD_DATA:
    data = load_data()
    np.random.shuffle(data)
    X_train,X_test,y_train,y_test = other_loading_data_proccess(data)

In [76]:
import torch
import torch.nn as nn

In [77]:
import torch.nn.functional as F

In [78]:
# class Test_Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 25 * 25, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 36)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 25 * 25)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [79]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 32, 5)
        self.fc1 = nn.Linear(16 * 25 * 25, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 36)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 25 * 25)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [80]:
device = torch.device('cuda')

In [81]:
model = Test_Model().to(device)

In [82]:
# preds = model(X_test.reshape(-1,1,112,112).float())

In [83]:
# preds[0]

In [84]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [85]:
EPOCHS = 3

In [86]:
loss_logs = []

In [87]:
from tqdm import tqdm

In [88]:
PROJECT_NAME = "Sign-Language-Recognition"

In [89]:
def test(net,X,y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in range(len(X)):
            real_class = torch.argmax(y[i]).to(device)
            net_out = net(X[i].view(-1,1,112,112).to(device).float())
            net_out = net_out[0]
            predictied_class = torch.argmax(net_out)
            if predictied_class == real_class:
                correct += 1
            total += 1
    return round(correct/total,3)

In [90]:
import wandb

In [91]:
len(os.listdir('./data/'))

36

In [92]:
import random

In [93]:
index = random.randint(0,29)
print(index)
wandb.init(project=PROJECT_NAME,name=NAME)
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item(),'accuracy':test(model,X_train,y_train)*100,'val_accuracy':test(model,X_test,y_test)*100,'pred':torch.argmax(preds[index]),'real':torch.argmax(y_batch[index])})
wandb.finish()

In [94]:
NAME = ""

In [95]:
BATCH_SIZE = 32

In [96]:
import os
import cv2
import torch
import numpy as np

In [97]:
def load_data(img_size=112):
    data = []
    index = -1
    labels = {}
    for directory in os.listdir('./data/'):
        index += 1
        labels[f'./data/{directory}/'] = [index,-1]
    print(len(labels))
    for label in labels:
        for file in os.listdir(label):
            filepath = label + file
            img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(img_size,img_size))
            img = img / 255.0
            data.append([
                np.array(img),
                labels[label][0]
            ])
            labels[label][1] += 1
    for _ in range(12):
        np.random.shuffle(data)
    print(len(data))
    np.save('./data.npy',data)
    return data

In [98]:
import torch

In [99]:
def other_loading_data_proccess(data):
    X = []
    y = []
    print('going through the data..')
    for d in data:
        X.append(d[0])
        y.append(d[1])
    print('splitting the data')
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    print('turning data to tensors')
    X_train = torch.from_numpy(np.array(X_train))
    y_train = torch.from_numpy(np.array(y_train))
    X_test = torch.from_numpy(np.array(X_test))
    y_test = torch.from_numpy(np.array(y_test))
    return [X_train,X_test,y_train,y_test]

In [100]:
REBUILD_DATA = True
if REBUILD_DATA:
    data = load_data()
    np.random.shuffle(data)
    X_train,X_test,y_train,y_test = other_loading_data_proccess(data)

In [101]:
import torch
import torch.nn as nn

In [102]:
import torch.nn.functional as F

In [103]:
# class Test_Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 25 * 25, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 36)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 25 * 25)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [104]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 32, 5)
        self.fc1 = nn.Linear(32 * 25 * 25, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 36)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 25 * 25)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [105]:
device = torch.device('cuda')

In [106]:
model = Test_Model().to(device)

In [107]:
# preds = model(X_test.reshape(-1,1,112,112).float())

In [108]:
# preds[0]

In [109]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [110]:
EPOCHS = 3

In [111]:
loss_logs = []

In [112]:
from tqdm import tqdm

In [113]:
PROJECT_NAME = "Sign-Language-Recognition"

In [114]:
def test(net,X,y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in range(len(X)):
            real_class = torch.argmax(y[i]).to(device)
            net_out = net(X[i].view(-1,1,112,112).to(device).float())
            net_out = net_out[0]
            predictied_class = torch.argmax(net_out)
            if predictied_class == real_class:
                correct += 1
            total += 1
    return round(correct/total,3)

In [115]:
import wandb

In [116]:
len(os.listdir('./data/'))

36

In [117]:
import random

In [118]:
index = random.randint(0,29)
print(index)
wandb.init(project=PROJECT_NAME,name=NAME)
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item(),'accuracy':test(model,X_train,y_train)*100,'val_accuracy':test(model,X_test,y_test)*100,'pred':torch.argmax(preds[index]),'real':torch.argmax(y_batch[index])})
wandb.finish()

In [119]:
import matplotlib.pyplot as plt

In [120]:
import pandas as pd

In [121]:
df = pd.Series(loss_logs)

In [122]:
df.plot.line(figsize=(12,6))

<AxesSubplot:>

In [123]:
test(model,X_test,y_test)

0.019

In [124]:
test(model,X_train,y_train)

0.021

In [125]:
preds

tensor([[ 0.3362, -3.4088, -2.2218,  ..., -2.3857,  1.0653,  0.3951],
        [ 2.1630, -1.6943, -1.6658,  ...,  1.4574,  6.3028, -1.1465],
        [-0.9519, -1.4624,  0.8735,  ..., -1.5962, -2.8113,  1.1790],
        ...,
        [ 4.1137, -2.7010, -4.4426,  ..., -1.4235,  7.6263, -2.2158],
        [ 0.1807, -2.2967,  5.6350,  ..., -0.1100, -2.0409,  1.1475],
        [-0.9329, -1.7657, -0.1154,  ..., -1.6887, -2.4095,  0.4703]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [126]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for i in range(len(X_test)):
        real_class = torch.argmax(y_test[i]).to(device)
        net_out = model(X_test[i].view(-1,1,112,112).to(device).float())
        net_out = net_out[0]
        predictied_class = torch.argmax(net_out)
#         print(predictied_class)
        if str(predictied_class) == str(real_class):
            correct += 1
        total += 1
print(round(correct/total,3))

In [127]:
total

628

In [128]:
for real,pred in zip(y_batch,preds):
    print(real)
    print(torch.argmax(pred))
    print('\n')

In [129]:
NAME = ""

In [130]:
BATCH_SIZE = 32

In [131]:
import os
import cv2
import torch
import numpy as np

In [132]:
def load_data(img_size=112):
    data = []
    index = -1
    labels = {}
    for directory in os.listdir('./data/'):
        index += 1
        labels[f'./data/{directory}/'] = [index,-1]
    print(len(labels))
    for label in labels:
        for file in os.listdir(label):
            filepath = label + file
            img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(img_size,img_size))
            img = img / 255.0
            data.append([
                np.array(img),
                labels[label][0]
            ])
            labels[label][1] += 1
    for _ in range(12):
        np.random.shuffle(data)
    print(len(data))
    np.save('./data.npy',data)
    return data

In [133]:
import torch

In [134]:
def other_loading_data_proccess(data):
    X = []
    y = []
    print('going through the data..')
    for d in data:
        X.append(d[0])
        y.append(d[1])
    print('splitting the data')
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    print('turning data to tensors')
    X_train = torch.from_numpy(np.array(X_train))
    y_train = torch.from_numpy(np.array(y_train))
    X_test = torch.from_numpy(np.array(X_test))
    y_test = torch.from_numpy(np.array(y_test))
    return [X_train,X_test,y_train,y_test]

In [135]:
REBUILD_DATA = True
if REBUILD_DATA:
    data = load_data()
    np.random.shuffle(data)
    X_train,X_test,y_train,y_test = other_loading_data_proccess(data)

In [136]:
import torch
import torch.nn as nn

In [137]:
import torch.nn.functional as F

In [138]:
# class Test_Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 25 * 25, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 36)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 25 * 25)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [139]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 32, 5)
        self.fc1 = nn.Linear(32 * 25 * 25, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 36)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 25 * 25)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [140]:
device = torch.device('cuda')

In [141]:
model = Test_Model().to(device)

In [142]:
# preds = model(X_test.reshape(-1,1,112,112).float())

In [143]:
# preds[0]

In [144]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [145]:
EPOCHS = 3

In [146]:
loss_logs = []

In [147]:
from tqdm import tqdm

In [148]:
PROJECT_NAME = "Sign-Language-Recognition"

In [149]:
def test(net,X,y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in range(len(X)):
            real_class = torch.argmax(y[i]).to(device)
            net_out = net(X[i].view(-1,1,112,112).to(device).float())
            net_out = net_out[0]
            predictied_class = torch.argmax(net_out)
            if predictied_class == real_class:
                correct += 1
            total += 1
    return round(correct/total,3)

In [150]:
import wandb

In [151]:
len(os.listdir('./data/'))

36

In [152]:
import random

In [153]:
index = random.randint(0,29)
print(index)
wandb.init(project=PROJECT_NAME,name=NAME)
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item(),'accuracy':test(model,X_train,y_train)*100,'val_accuracy':test(model,X_test,y_test)*100,'pred':torch.argmax(preds[index]),'real':torch.argmax(y_batch[index])})
wandb.finish()